In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/joaquin/Documents/GitHub/skforecast


In [2]:
import os
import re

import numpy as np
import pandas as pd
import pytest

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model

# Export torch keras backend
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

2025-07-29 12:41:18.724900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753785678.743025   44658 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753785678.748588   44658 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753785678.763225   44658 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753785678.763244   44658 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753785678.763246   44658 computation_placer.cc:177] computation placer alr

In [3]:
series = pd.DataFrame(np.arange(10), columns=["l1"])
model = create_and_compile_model(
            series=series, 
            levels="l1",    
            lags=20,           
            steps=3,              
            recurrent_layer="LSTM",
            recurrent_units=100,
            recurrent_layers_kwargs={"activation": "relu"},
            dense_units=[128, 64],
            dense_layers_kwargs={"activation": "relu"},
            output_dense_layer_kwargs={"activation": "linear"},
            compile_kwargs={"optimizer": Adam(learning_rate=0.01), "loss": MeanSquaredError()},
        )

keras version: 3.10.0
Using backend: tensorflow
tensorflow version: 2.19.0



In [4]:
y_array = np.arange(10)
forecaster = ForecasterRnn(model, levels="l1", lags=20)

err_msg = re.escape(
    f"The maximum lag ({forecaster.max_lag}) must be less than the length "
    f"of the series minus the maximum of steps ({len(series)-forecaster.max_step})."
)
forecaster._create_lags(y=y_array)

/home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: negative dimensions are not allowed